## Init

In [2]:
import numpy as np
import polars as pl

In [6]:
DATAPATH = './data'
train = pl.read_csv(f'{DATAPATH}/train.csv')
test = pl.read_csv(f'{DATAPATH}/test.csv')
submission_sample = pl.read_csv(f'{DATAPATH}/gender_submission.csv')

## Data Exploration

In [59]:
train_explore = train.clone()

In [63]:
# train_explore.head()
train_explore.describe()

describe,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",891.0,891.0,891.0,"""891""",891.0,"""891""","""891"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. An…","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""25%""",223.0,0.0,2.0,null,null,20.0,0.0,0.0,null,7.8958,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""75%""",669.0,1.0,3.0,null,null,38.0,1.0,0.0,null,31.0,null,null
"""max""",891.0,1.0,3.0,"""van Melkebeke,…","""male""",80.0,8.0,6.0,"""WE/P 5735""",512.3292,"""T""","""S"""


In [108]:
'''
Name
Ticket
Cabin
'''
(
    train_explore
    .select(
        pl.col('Name').str.split(by=', ').apply(lambda x: x[0]).alias('LastName'),
        pl.col('Name').str.split(by=', ').apply(lambda x: x[1]).str.split(by=' ').apply(lambda x: x[0]).alias('Honorific')
    )
)

# train_explore.select('Name', pl.col('Ticket')).sort('Ticket').head(5)

# train_explore.select('Cabin', pl.col('Ticket')).sort('Cabin').head(5)
# train_explore.groupby('Cabin').agg(pl.n_unique('PassengerId').alias('counts')).sort('counts', descending=True)
# train_explore.filter(pl.col('Cabin')=='C23 C25 C27')
# display(train_explore.groupby('Ticket').agg(pl.n_unique('Cabin').alias('cabincount')).filter(pl.col('cabincount') > 1).head(5))
# train_explore.filter(pl.col('Ticket') == '17421')

Ticket,cabincount
str,u32
"""13502""",3
"""110465""",2
"""PC 17760""",3
"""PC 17582""",2
"""PC 17572""",2


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
307,1,1,"""Fleming, Miss.…","""female""",null,0,0,"""17421""",110.8833,null,"""C"""
551,1,1,"""Thayer, Mr. Jo…","""male""",17.0,0,2,"""17421""",110.8833,"""C70""","""C"""
582,1,1,"""Thayer, Mrs. J…","""female""",39.0,1,1,"""17421""",110.8833,"""C68""","""C"""
699,0,1,"""Thayer, Mr. Jo…","""male""",49.0,1,1,"""17421""",110.8833,"""C68""","""C"""


In [57]:
def get_features_to_outcome_effect(df: pl.DataFrame, group_cols: list[str], outcome_col: str) -> None:
    for col in group_cols:
        if df[col].dtype in [pl.Float32, pl.Float64]:
            df = (
                df
                .with_columns(pl.col(col).qcut(5).alias(col))
            )
    return df.groupby(group_cols).agg(pl.mean(outcome_col)).sort(group_cols, descending=False)

# single_variable_relation_to_outcome(train_explore, ['Sex'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Pclass'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Age'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['SibSp'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Parch'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Fare'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Embarked'], 'Survived')

## Transform data

In [109]:
from sklearn.pipeline import Pipeline
train_transformed = train.clone()

In [118]:
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.decomposition import PCA, TruncatedSVD
union = FeatureUnion([
    ("pca", PCA(n_components=1)),
    ("svd", TruncatedSVD(n_components=2))
])
X = [[0., 1., 3], [2., 2., 5]]
union.fit_transform(X)

array([[ 3.03954967,  0.87243213],
       [ 5.72586357, -0.46312679]])